<a href="https://colab.research.google.com/github/returaj/cs6910/blob/assginment1_akash/q_3_random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 48.9 MB/s 
     |████████████████████████████████| 144 kB 50.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=54746d66ff153706a3c7487ee2e9327285173a8e20892a8541de07cd5c18921c
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/DLASSIGNMENT/activation_func.py .
import activation_func
!cp drive/MyDrive/DLASSIGNMENT/loss_func.py .
import loss_func
!cp drive/MyDrive/DLASSIGNMENT/runner.py .
!cp drive/MyDrive/DLASSIGNMENT/feed_forward_nn.py .
!cp drive/MyDrive/DLASSIGNMENT/measure.py .
!cp drive/MyDrive/DLASSIGNMENT/optimizer.py .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from keras.datasets import fashion_mnist

import wandb

from feed_forward_nn import FNN
from measure import accuracy
from optimizer import SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam
from activation_func import Sigmoid, Relu, Tanh
from loss_func import CrossEntropy, MeanSquaredError
from sklearn.model_selection import train_test_split

In [9]:
import numpy as np

np.random.seed(2)
class Runner_r(object):
  def __init__(self, isFashionMnistData=True):
    if isFashionMnistData:
      self.initialize_fashion_mnist_data()
    else:
      self.initialize_data()

  def initialize_fashion_mnist_data(self):
    (X, self.y), (X_test, self.y_test) = fashion_mnist.load_data()
    self.X = np.array([x.flatten() for x in X]) / 255
    self.X_test = np.array([x.flatten() for x in X_test]) / 255
    

  def initialize_data(self):
    raise NotImplementedError("Please implement this method if you need other dataset.")

  @staticmethod
  def get_loss_function(key):
    mapper = {
      "cross_entropy": CrossEntropy,
      "mse": MeanSquaredError
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_activation_function(key):
    mapper = {
      "sigmoid": Sigmoid,
      "relu": Relu,
      "tanh": Tanh
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_optimizer(key):
    mapper = {
      "sgd": SGD,
      "momentum_gd": MomentumGD,
      "nesterov_gd": NesterovGD,
      "rmsprop": Rmsprop,
      "adam": Adam,
      "nadam": Nadam
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def train(X_train, y_train,params,do_val, wandb_log, print_after=500):
    """
    X_train: (batch_size(B), data_size(N))
    y_train: (batch_size(B))
    params: dict(
      batch_size: int,
      epochs: int,
      alpha: float,
      optimizer: One of (SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam),
      hidden_layers_size: list(layer_size),
      act_func: One of (Sigmoid, Relu, Tanh),
      reg: float,
      init: One of (random, xavier),
      loss_func: One of (CrossEntropy, MeanSquaredError)
    )
    """
    if do_val:
      X_train, X_val, y_train, y_val = train_test_split(
          X_train, y_train, test_size=0.1, random_state=10)

    data_size, input_size = X_train.shape
    output_size = 10
    batch_size, epochs = params['batch_size_r'], params['epochs_r']
    act_func, loss_func = params['act_func_r'](), params['loss_func_r']()
    model = FNN(
      input_size         = input_size,
      output_size        = output_size,
      hidden_layers_size = params['hidden_layers_size_r'],
      act_func           = act_func,
      reg                = params['reg_r'],
      init               = params['init_r'],
      loss_func          = loss_func,
    )
    optimizer = params['optimizer_r'](model, params['alpha_r'])
    for ep in range(1, epochs+1):
      ids = np.arange(data_size)
      np.random.shuffle(ids)
      start, end = 0, batch_size
      while end > start:
        x, y = X_train[ids[start:end]], y_train[ids[start:end]]
        optimizer.optimize(x, y)
        start, end = end, min(data_size, end+batch_size)
      # log
      train_loss = loss_func.error(X_train, y_train, model)
      estimate_y_train = Runner_r.predict(X_train, model)
      train_acc = accuracy(estimate_y_train, y_train)
      val_loss, val_acc = "NotDefined", "NotDefined"
      if do_val:
        val_loss = loss_func.error(X_val, y_val, model)
        estimate_y_val = Runner_r.predict(X_val, model)
        val_acc = accuracy(estimate_y_val, y_val)

      Runner_r.logger(train_loss,train_acc,val_loss,val_acc,ep,wandb_log)
    #estimate_y_test = Runner.predict(X_test,model)
    #test_acc = accuracy(estimate_y_test,Y_test)
    #wandb.log({"test_accuracy":test_acc})
    return model
  @staticmethod
  def test(model,X_test,Y_test):
    estimate_y_test = Runner_r.predict(X_test,model)
    test_acc = accuracy(estimate_y_test,Y_test)
    wandb.log({"test_accuracy_random":test_acc})
    return model


  def logger(train_loss, train_acc, val_loss, val_acc,step,wandb_log):
    # print(f"TrainingLoss: {train_loss}, TrainingAccuracy: {train_acc}")
    # print(f"ValidationLoss: {val_loss}, ValidationAccuracy: {val_acc}")
    if wandb_log:
      wandb.log({
        "epochs_random": step,
        "Training_Loss_random": train_loss,
        "Training_Accuracy_random": train_acc,
        "Validation_Loss_random": val_loss,
        "Validation_Accuracy_random": val_acc,  
      }
      )




  @staticmethod
  def predict(X, model):
    """
    X: (batch_size(B), data_size(N))
    return np.array of size: (batch_size(B),)
    """
    prob = model.forward(X)[-1]
    return np.argmax(prob, axis=1)


def run_wandb():
  wandb.init(project="CS6910_ASSIGNMENT", entity="cs21m003_cs21d406")
  config = wandb.config
  loss_name = "ce" if config.loss_func_r == "cross_entropy" else "mse"
  wandb.run.name=f"e_{config.epochs_r}_bs_{config.batch_size_r}_hl_{config.hidden_layers_r}_hn_{config.hidden_nodes_r}_init_{config.weight_init_r}_ac_{config.act_func_r}_ls_{loss_name}_opt_{config.optimizer_r}_rate_{config.learning_rate_r}_reg_{config.reg_r}_random"
  hidden_layers_size = [config.hidden_nodes_r] * config.hidden_layers_r
  runner =Runner_r()
  params = {
    "batch_size_r"        : config.batch_size_r,
    "epochs_r"            : config.epochs_r,
    "alpha_r"             : config.learning_rate_r,
    "optimizer_r"         : runner.get_optimizer(config.optimizer_r),
    "hidden_layers_size_r": hidden_layers_size,
    "act_func_r"          : runner.get_activation_function(config.act_func_r),
    "reg_r"               : config.reg_r,
    "init_r"              : config.weight_init_r,
    "loss_func_r"         : runner.get_loss_function(config.loss_func_r),
  }
  model=runner.train(runner.X, runner.y, params, do_val=True, wandb_log=True)
  runner.test(model,runner.X_test,runner.y_test)

def do_hyperparameter_search_using_wandb():
  sweep_config = {
    "name": "Random Sweep",
    "method": "random",
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "epochs_r": {"values": [5, 10]}, 
      "batch_size_r": {"values": [16, 32, 64]}, 
      "hidden_layers_r": {"values": [3, 4, 5]}, 
      "hidden_nodes_r": {"values": [32, 64, 128]},
      "reg_r": {"values": [0, 0.0005, 0.5]},
      "weight_init_r": {"values": ['random', 'xavier']} , 
      "act_func_r": {"values": ["sigmoid", "tanh", "relu"]}, 
      "loss_func_r": {"values": ["cross_entropy"]}, 
      "learning_rate_r": {"values": [1e-3, 1e-4]},   
      "optimizer_r": {"values": ["sgd", "momentum_gd", "nesterov_gd", "rmsprop", "adam", "nadam"]}, 
    }
  }
  
  sweep_id = wandb.sweep(sweep_config, project = "CS6910_ASSIGNMENT",entity='cs21m003_cs21d406')
  wandb.agent(sweep_id, function=run_wandb, count=50)


if __name__ == '__main__':
  do_hyperparameter_search_using_wandb()  

Create sweep with ID: jhcj8bur
Sweep URL: https://wandb.ai/cs21m003_cs21d406/CS6910_ASSIGNMENT/sweeps/jhcj8bur


wandb: Agent Starting Run: dkayozsz with config:
wandb: 	act_func_r: sigmoid
wandb: 	batch_size_r: 64
wandb: 	epochs_r: 10
wandb: 	hidden_layers_r: 3
wandb: 	hidden_nodes_r: 64
wandb: 	learning_rate_r: 0.0001
wandb: 	loss_func_r: cross_entropy
wandb: 	optimizer_r: nadam
wandb: 	reg_r: 0
wandb: 	weight_init_r: xavier
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Training_Accuracy_random,▂▆▇▄▇█▄█▁█
Training_Loss_random,███▇▇▇▇▆▅▁
Validation_Accuracy_random,█▃▂▇▂▁▇▁▂▁
Validation_Loss_random,▇████▇▇▆▅▁
epochs_random,▁▂▃▃▄▅▆▆▇█
test_accuracy_random,▁
Training_Accuracy_random,0.1005
Training_Loss_random,2.29925
Validation_Accuracy_random,0.0955
Validation_Loss_random,2.29935
epochs_random,10


wandb: Agent Starting Run: fmslk8ge with config:
wandb: 	act_func_r: sigmoid
wandb: 	batch_size_r: 32
wandb: 	epochs_r: 10
wandb: 	hidden_layers_r: 3
wandb: 	hidden_nodes_r: 32
wandb: 	learning_rate_r: 0.0001
wandb: 	loss_func_r: cross_entropy
wandb: 	optimizer_r: adam
wandb: 	reg_r: 0
wandb: 	weight_init_r: xavier
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Training_Accuracy_random,▁▁▁▁▁▁▁▁█▆
Training_Loss_random,████████▇▁
Validation_Accuracy_random,▁▁▁▁▁▁▁▁█▆
Validation_Loss_random,████████▇▁
epochs_random,▁▂▃▃▄▅▆▆▇█
test_accuracy_random,▁
Training_Accuracy_random,0.20341
Training_Loss_random,2.12224
Validation_Accuracy_random,0.19633
Validation_Loss_random,2.12179
epochs_random,10


...

Training_Accuracy_random,█▇▄▁▇▂▃▂█▂
Training_Loss_random,▂▄█▂▁▅▄▆▄▇
Validation_Accuracy_random,▁▂▅█▂▇▆▇▁▇
Validation_Loss_random,▁▄█▁▂▅▃▆▅▅
epochs_random,▁▂▃▃▄▅▆▆▇█
test_accuracy_random,▁
Training_Accuracy_random,0.0997
Training_Loss_random,2.32193
Validation_Accuracy_random,0.10267
Validation_Loss_random,2.31996
epochs_random,10


wandb: Agent Starting Run: k4fza8v3 with config:
wandb: 	act_func_r: tanh
wandb: 	batch_size_r: 16
wandb: 	epochs_r: 5
wandb: 	hidden_layers_r: 4
wandb: 	hidden_nodes_r: 64
wandb: 	learning_rate_r: 0.0001
wandb: 	loss_func_r: cross_entropy
wandb: 	optimizer_r: momentum_gd
wandb: 	reg_r: 0.5
wandb: 	weight_init_r: xavier
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Training_Accuracy_random,▁▁▁▁▁
Training_Loss_random,█▁▁▁▁
Validation_Accuracy_random,▁▁▁▁▁
Validation_Loss_random,█▁▁▁▁
epochs_random,▁▃▅▆█
test_accuracy_random,▁
Training_Accuracy_random,0.1005
Training_Loss_random,2.30258
Validation_Accuracy_random,0.0955
Validation_Loss_random,2.30268
epochs_random,5


wandb: Agent Starting Run: a53ynlba with config:
wandb: 	act_func_r: tanh
wandb: 	batch_size_r: 64
wandb: 	epochs_r: 10
wandb: 	hidden_layers_r: 4
wandb: 	hidden_nodes_r: 64
wandb: 	learning_rate_r: 0.001
wandb: 	loss_func_r: cross_entropy
wandb: 	optimizer_r: nadam
wandb: 	reg_r: 0
wandb: 	weight_init_r: random
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Training_Accuracy_random,▄█▁▂▃▆█▂▂█
Training_Loss_random,▃▁▅▅▆▄█▆▄▆
Validation_Accuracy_random,▅▁█▇▆▃▁▇▇▁
Validation_Loss_random,▃▁▃▃▆▃█▃▃▅
epochs_random,▁▂▃▃▄▅▆▆▇█
test_accuracy_random,▁
Training_Accuracy_random,0.1005
Training_Loss_random,2.31159
Validation_Accuracy_random,0.0955
Validation_Loss_random,2.31136
epochs_random,10
